Twitter Sentiment Analysis

Import the libraries that will be usedin this project.

In [142]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 1000

Read in the twitter data into a panda dataframe. I originally ran into an encoding issue and had to save the csv in UTF-8. The csv file does not include a column header row, so add those in manually.

In [143]:
df = pd.read_csv("twitter_data.csv", header=None, names=["sentiment", "tweet_id", "date", "query", "user", "tweet"])

In [144]:
df.head()

,sentiment,tweet_id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


Remove columns that dont seem useful for sentiment analysis

In [145]:
df.drop(["tweet_id", "date", "query", "user"], axis=1, inplace=True)
df.head()

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


DO NOT KEEP THIS IN THE PROJECT!!!
DROPPING A BUNCH OF ROWS TO SPEED UP PREPROCESSING WHILE TESTING!!!

In [146]:
df = df.sample(10000)
df.shape

(10000, 2)

Perform data preprocessing

In [147]:
import re

def processTweet(tweet):
    tweet = re.sub("[@|#]\w+\S","", tweet) # remove @usernames and #hashtags
    tweet = re.sub("http[s]?://[\S]+", '', tweet) # remove urls 
    tweet = re.sub(r"(.)\1\1+",r"\1\1", tweet) # remove letters that repeat more than 2 times
    return tweet

In [148]:
df['tweet'] = df['tweet'].map(lambda tweet: processTweet(tweet))

df.head()

,sentiment,tweet
1486752,4,Ohh check out this twitter item..lol. another thing for peoples ego's
58470,0,omg same! im bout to do french x
784187,0,I still haven't gone to sleep.
1092241,4,Soo many great twitter friends!!
1050217,4,will have 2 play vodka checkers


In [149]:
from sklearn.model_selection import train_test_split

y = df['sentiment'].map({0: 0, 4: 1})
X = df.drop(['sentiment'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [150]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()

train_vector = count_vector.fit_transform(X_train['tweet'])
test_vector = count_vector.transform(X_test['tweet'])

In [151]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(train_vector, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [152]:
predictions = naive_bayes.predict(test_vector)

In [153]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[978 296]
 [321 905]]
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      1274
           1       0.75      0.74      0.75      1226

    accuracy                           0.75      2500
   macro avg       0.75      0.75      0.75      2500
weighted avg       0.75      0.75      0.75      2500

0.7532


BELOW HERE STARTING RNN

In [154]:
from keras.preprocessing import sequence, text

vocabulary_size = 5000

tokenizer = text.Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(X_train['tweet'])

train_token = tokenizer.texts_to_sequences(X_train['tweet'])
test_token = tokenizer.texts_to_sequences(X_test['tweet'])

max_words = 50
train_padded = sequence.pad_sequences(train_token, maxlen=max_words)
test_padded = sequence.pad_sequences(test_token, maxlen=max_words)

In [187]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Activation

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())



# model.add(Dense(512, input_dim=50))
# model.add(Activation("relu"))
# model.add(Dropout(0.5))
# model.add(Dense(2))
# model.add(Activation("softmax"))
# model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 50, 32)            160000    
_________________________________________________________________
lstm_15 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [188]:
loss_func = 'binary_crossentropy'
# loss_func = 'categorical_crossentropy'
model.compile(loss=loss_func, 
             optimizer='adam', 
             metrics=['accuracy'])

In [189]:
batch_size = 64
num_epochs = 3
# X_valid, y_valid = train_padded[:batch_size], y_train[:batch_size]
# X_train2, y_train2 = train_padded[batch_size:], y_train[batch_size:]
# model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
model.fit(train_padded, y_train, validation_data=(test_padded, y_test), batch_size=batch_size, epochs=num_epochs)

Train on 7500 samples, validate on 2500 samples
Epoch 1/3
7500/7500 [==============================] - 6s 832us/step - loss: 0.6388 - acc: 0.6215 - val_loss: 0.5474 - val_acc: 0.7352
Epoch 2/3
7500/7500 [==============================] - 4s 555us/step - loss: 0.4575 - acc: 0.7879 - val_loss: 0.5186 - val_acc: 0.7468
Epoch 3/3
7500/7500 [==============================] - 4s 556us/step - loss: 0.3578 - acc: 0.8492 - val_loss: 0.5413 - val_acc: 0.7412


In [190]:
scores = model.evaluate(test_padded, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.7412
